# Задание по программированию: Размещение баннеров
Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [23]:
import csv
with open('checkins.dat') as dat_file, open('checkins_csv.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)
            

In [31]:
import pandas as pd
from sklearn.cluster import MeanShift

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание:на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [47]:
data = pd.read_csv('checkins_csv.csv')
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18


In [48]:
data.shape
data = data.drop(['id', 'user_id', 'venue_id', 'created_at'], axis = 1)

In [49]:
clustering = MeanShift(bandwidth = 0.1, n_jobs = -1, min_bin_freq = 15)
clustering.fit(data[:100000])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=15,
          n_jobs=-1, seeds=None)

In [54]:
cluster_centers = clustering.cluster_centers_
cluster_centers[:10]

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       [  41.87824378,  -87.62984336],
       [  37.68868157, -122.40933037],
       [  38.88616522,  -77.04878333],
       [  33.35734456, -111.82265411],
       [  33.76663623,  -84.39328918],
       [  42.36321864,  -71.07368761],
       [  47.60624472, -122.33204383]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.



In [67]:
from numpy.linalg import norm
office_coordinates = {'Los Angeles': [33.751277, -118.188740],
                      'Miami': [25.867736, -80.324116],
                      'London': [51.503016, -0.075479],
                      'Amsterdam': [52.378894, 4.885084],
                      'Beijing': [39.366487, 117.036146],
                      'Sydney': [-33.868457, 151.205134]}
distances = []
for index, item in enumerate(cluster_centers):
    for key, value in office_coordinates.items():
        distances.append([np.sqrt(sum((item - value)**2)), index])
distances.sort()
distances = distances[:20]

In [68]:
for item in distances:
    print(cluster_centers[item[1]])

[-33.86063043 151.20477593]
[52.37296399  4.89231722]
[ 25.84567226 -80.3188906 ]
[51.50299126 -0.12553729]
[  33.80987796 -118.14892381]
[ 25.78581242 -80.21793804]
[-34.00190615 151.12806905]
[ 25.70534972 -80.28342874]
[ 26.01009825 -80.19999059]
[-33.9522629 151.0321372]
[  33.88832534 -118.04892817]
[  33.87298601 -118.36209115]
[  33.97257482 -118.16837067]
[51.42676329 -0.30373207]
[52.388501    4.63376547]
[51.5741517  0.1838708]
[ 26.13884379 -80.33434684]
[52.2644  4.6347]
[51.50647877 -0.36517727]
[  33.98393587 -118.00740497]


In [70]:
# Ответ: 
cluster_centers[distances[0][1]]

array([-33.86063043, 151.20477593])

In [71]:
distances

[[0.007834758163107856, 420],
 [0.009353316185992226, 370],
 [0.022674066158385495, 419],
 [0.05005829482278787, 58],
 [0.07084773242719973, 51],
 [0.13410903336184654, 29],
 [0.154102829806012, 2216],
 [0.16740596425035326, 167],
 [0.18887596060185083, 92],
 [0.19222726571703244, 2214],
 [0.19577945647763628, 87],
 [0.21181053682436798, 42],
 [0.2222332907317907, 291],
 [0.24065314956287862, 802],
 [0.2515020867623528, 1003],
 [0.268928627287111, 2289],
 [0.2713007595066735, 320],
 [0.27531985669762415, 2273],
 [0.28971896117213125, 1006],
 [0.2949788868004569, 119]]